<h1>Projeto De Previsao de Compras</h1>
<h2>Versao.: 1.3</h2>
<h2>Data.:   15/02/2019  16:20</h2>

<h3>
    TODO:<br/>
    <s>1º Reajustar a média móvel;</s><br/>
    <s>2º Calibrar com o auto-arima os modelos arima e sarima;</s><br/>
    <s>3º Gerar o erro quadrático para cada modelo;</s><br/>
    <s>4º Fazer a seleção das quantidades a serem compradas do modelo que contiver o menor erro;</s><br/>
       5º inserir indices a partir de 01/01/2016;<br/>
       6º estudar casos com quantidades = 0 (interpolação ou 0);<br/>
       7º Rodar o projeto com todos os itens.<br/>
</h3>

In [117]:
import timeit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from datetime import datetime
from math import sqrt, floor, ceil
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.api import SimpleExpSmoothing, Holt, ExponentialSmoothing
from statsmodels.tsa.arima_model import ARIMA
from pmdarima.arima import auto_arima

%matplotlib inline

try:
    dateparse = lambda dates: pd.datetime.strptime(dates, '%d.%m.%Y %H:%M:%S')
    dataFrame = pd.read_csv(r"C:\Macros\biotropic.CSV", sep=";", parse_dates=['dt_venda'], 
                        index_col='dt_venda', date_parser=dateparse)
except:
    dateparse = lambda dates: pd.datetime.strptime(dates, '%d/%m/%Y %H:%M:%S')
    dataFrame = pd.read_csv(r"C:\Macros\biotropic.CSV", sep=";", parse_dates=['dt_venda'], 
                       index_col='dt_venda', date_parser=dateparse)
    


In [142]:
dataFrame[(dataFrame['cd_mercadoria'] == 365573) & (dataFrame['cd_empresa'] == 1)].info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 35 entries, 2016-01-01 to 2019-01-01
Data columns (total 3 columns):
cd_empresa       35 non-null int64
cd_mercadoria    35 non-null int64
qt_vendida       35 non-null int64
dtypes: int64(3)
memory usage: 1.1 KB


In [119]:
def buscarDataAtual():
    data_atual = datetime.today()
    data_em_texto = '{}-{}-{}'.format(data_atual.year, data_atual.month, '01')
    return data_em_texto

In [120]:
def adicionarIndicesFaltantes(df):
    idx = pd.date_range('2016-01-01', dataAtual, freq='MS')
    df.index = pd.DatetimeIndex(df.index)
    df = df.reindex(idx, fill_value=0)
    return df

In [121]:
def suavizacaoExponencialSimples(dataFrameMercadoria):
    
    totalLinhas = dataFrameMercadoria.shape[0]
    quantidadeLinhasTreino = floor(totalLinhas-((totalLinhas*20)/100))

    ts = dataFrameMercadoria['qt_vendida']
    
    train = ts[0:quantidadeLinhasTreino]
    teste = ts[quantidadeLinhasTreino:len(ts)-1] 
    
    y_hat_avg = pd.DataFrame(teste.copy())
    model_fit = SimpleExpSmoothing(np.asarray(train)).fit(smoothing_level=0.6,optimized=False)
    y_hat_avg['SES'] = model_fit.forecast(len(teste)) 
    
    for i, hatAvg in y_hat_avg.iterrows():
        dataFrameMercadoria.set_value(y_hat_avg[i:i+1].index.date,'SES',y_hat_avg[i:i+1])
    
    train = ts[0:len(ts)-1]
    teste = ts[len(ts)-1:len(ts)]
    
    y_hat_avg = pd.DataFrame(teste.copy())
    model_fit = SimpleExpSmoothing(np.asarray(train)).fit(smoothing_level=0.6,optimized=False)
    y_hat_avg['SES'] = model_fit.forecast(len(teste))
    
    for i, hatAvg in y_hat_avg.iterrows():
        dataFrameMercadoria.set_value(y_hat_avg[i:i+1].index.date,'SES',y_hat_avg[i:i+1])
    
    return dataFrameMercadoria

In [122]:
def holtLinear(dataFrameMercadoria):
    
    totalLinhas = dataFrameMercadoria.shape[0]
    quantidadeLinhasTreino = floor(totalLinhas-((totalLinhas*20)/100))

    ts = dataFrameMercadoria['qt_vendida']
    
    train = ts[0:quantidadeLinhasTreino]
    teste = ts[quantidadeLinhasTreino:len(ts)-1] 
    
    y_hat_avg = pd.DataFrame(teste.copy())
    fit1 = Holt(np.asarray(train)).fit(smoothing_level = 0.6,smoothing_slope = 0.1)
    y_hat_avg['Holt_linear'] = fit1.forecast(len(teste))
    
    for i, hatAvg in y_hat_avg.iterrows():
        dataFrameMercadoria.set_value(y_hat_avg[i:i+1].index.date,'Holt_linear',y_hat_avg[i:i+1])
        
    train = ts[0:len(ts)-1]
    teste = ts[len(ts)-1:len(ts)]
    
    y_hat_avg = pd.DataFrame(teste.copy())
    fit1 = Holt(np.asarray(train)).fit(smoothing_level = 0.6,smoothing_slope = 0.1)
    y_hat_avg['Holt_linear'] = fit1.forecast(len(teste))
    
    for i, hatAvg in y_hat_avg.iterrows():
        dataFrameMercadoria.set_value(y_hat_avg[i:i+1].index.date,'Holt_linear',y_hat_avg[i:i+1])
    
    return dataFrameMercadoria

In [123]:
def holtWinter(dataFrameMercadoria):
    
    totalLinhas = dataFrameMercadoria.shape[0]
    quantidadeLinhasTreino = floor(totalLinhas-((totalLinhas*20)/100))

    ts = dataFrameMercadoria['qt_vendida']
    
    train = ts[0:quantidadeLinhasTreino]
    teste = ts[quantidadeLinhasTreino:len(ts)-1] 
    
    y_hat_avg = pd.DataFrame(teste.copy())
    fit1 = ExponentialSmoothing(np.asarray(train) ,seasonal_periods=12,trend='add', seasonal='add' ).fit()
    y_hat_avg['Holt_Winter'] = fit1.forecast(len(teste))
    
    for i, hatAvg in y_hat_avg.iterrows():
        dataFrameMercadoria.set_value(y_hat_avg[i:i+1].index.date,'Holt_Winter',y_hat_avg[i:i+1])
        
    train = ts[0:len(ts)-1]
    teste = ts[len(ts)-1:len(ts)]
    
    y_hat_avg = pd.DataFrame(teste.copy())
    fit1 = ExponentialSmoothing(np.asarray(train) ,seasonal_periods=12,trend='add', seasonal='add' ).fit()
    y_hat_avg['Holt_Winter'] = fit1.forecast(len(teste))
    
    for i, hatAvg in y_hat_avg.iterrows():
        dataFrameMercadoria.set_value(y_hat_avg[i:i+1].index.date,'Holt_Winter',y_hat_avg[i:i+1])

    return dataFrameMercadoria
    

In [124]:
def arima(dataFrameMercadoria):
    
    totalLinhas = dataFrameMercadoria.shape[0]
    quantidadeLinhasTreino = floor(totalLinhas-((totalLinhas*20)/100))

    ts = dataFrameMercadoria['qt_vendida']
    
    train = ts[0:quantidadeLinhasTreino]
    teste = ts[quantidadeLinhasTreino:len(ts)-1] 
    
    modelo_auto = auto_arima(ts, m = 12, seasonal = True, trace = False)
    
    arimaOrder = modelo_auto.order
    
    y_hat_avg = pd.DataFrame(teste.copy())
    fit1 = ARIMA(train, order=arimaOrder).fit(disp=0)
    forecast, stderr, conf = fit1.forecast(len(teste))
    y_hat_avg['ARIMA'] = forecast
    
    for i, hatAvg in y_hat_avg.iterrows():
        dataFrameMercadoria.set_value(y_hat_avg[i:i+1].index.date,'ARIMA',y_hat_avg[i:i+1])
    
    train = ts[0:len(ts)-1]
    teste = ts[len(ts)-1:len(ts)]
    
    y_hat_avg = pd.DataFrame(teste.copy())
    fit1 = ARIMA(train, order=arimaOrder).fit(disp=0)
    forecast, stderr, conf = fit1.forecast(len(teste))
    y_hat_avg['ARIMA'] = forecast
    
    for i, hatAvg in y_hat_avg.iterrows():
        dataFrameMercadoria.set_value(y_hat_avg[i:i+1].index.date,'ARIMA',y_hat_avg[i:i+1])

    return dataFrameMercadoria
    
    

In [125]:
def sarima(dataFrameMercadoria):
    
    totalLinhas = dataFrameMercadoria.shape[0]
    quantidadeLinhasTreino = floor(totalLinhas-((totalLinhas*20)/100))

    ts = dataFrameMercadoria['qt_vendida']
    
    train = ts[0:quantidadeLinhasTreino]
    teste = ts[quantidadeLinhasTreino:len(ts)-1]
    
    modelo_auto = auto_arima(ts, m = 12, seasonal = True, trace = False)
    
    sarimaOrder = modelo_auto.order
    seasonalOrder = modelo_auto.seasonal_order
    
    y_hat_avg = pd.DataFrame(teste.copy())
    fit1 = sm.tsa.statespace.SARIMAX(train, order=sarimaOrder,seasonal_order=seasonalOrder, 
                                 enforce_stationarity=False, enforce_invertibility=False).fit()
    y_hat_avg['SARIMA'] = fit1.forecast(len(teste))
    
    for i, hatAvg in y_hat_avg.iterrows():
        dataFrameMercadoria.set_value(y_hat_avg[i:i+1].index.date,'SARIMA',y_hat_avg[i:i+1])
        
    train = ts[0:len(ts)-1]
    teste = ts[len(ts)-1:len(ts)]
    
    y_hat_avg = pd.DataFrame(teste.copy())
    fit1 = sm.tsa.statespace.SARIMAX(train, order=sarimaOrder,seasonal_order=seasonalOrder, 
                                 enforce_stationarity=False, enforce_invertibility=False).fit()
    y_hat_avg['SARIMA'] = fit1.forecast(len(teste))
    
    for i, hatAvg in y_hat_avg.iterrows():
        dataFrameMercadoria.set_value(y_hat_avg[i:i+1].index.date,'SARIMA',y_hat_avg[i:i+1])
        
    return dataFrameMercadoria
    

In [126]:
def calcularMediaMovel(dataFrameMercadoria):
    j = 1
    while j <= 12:
        
        mediasMoveisCalculada = dataFrameMercadoria['qt_vendida'].rolling(j).mean()
        
        data_minima = mediasMoveisCalculada.index.min()+1
        data_maxima = mediasMoveisCalculada.index.max()
        
        date_index = pd.date_range(data_minima, data_maxima, freq="MS")
        serie = pd.Series(index=date_index)
        
        for i, iValue in serie.iteritems():
            serie[i] = mediasMoveisCalculada[i-1]
          
        dataFrameMercadoria['MM'+str(j)] = serie;

        j += 1
    return dataFrameMercadoria

In [127]:
def calcularErro(dataFrameMercadoria):
    aVendida = df.iloc[-7:-1, 2]
    result = list()
    j = 1
    while j < 17:
        modelo = df.columns[j+2]
        aMM = df.iloc[-7:-1:, j+2].values
        previsao = df.iloc[-1, j+2]
        cd = max(df.cd_empresa.unique())
        produto = max(df.cd_mercadoria.unique())
        erroMM = sqrt(mean_squared_error(aVendida, aMM))
        if len(result) > 0:
            if  result[4] > erroMM:
                result = [cd, produto, modelo, previsao, erroMM]
        elif len(result) == 0:
            result = [cd, produto, modelo, previsao, erroMM]
        j += 1

    return result


In [138]:
#inicio = timeit.default_timer()

#arrayMercadoria = dataFrame['cd_mercadoria'].unique()
#arrayEmpresa = [1, 4, 5]
arrayMercadoria = [365573]
arrayEmpresa = [1]

arrayDeDataFrame = []

dataAtual = buscarDataAtual()
dataFrame = dataFrame[dataFrame.index != dataAtual]
for emp in arrayEmpresa:
    i = 0
    for i, mer in enumerate(arrayMercadoria):
        df = dataFrame[(dataFrame['cd_mercadoria'] == mer) & 
                      (dataFrame['cd_empresa'] == emp)].sort_values(by=['dt_venda'], ascending=True)
        print(mer)
        if len(df) > 7:
            df = adicionarIndicesFaltantes(df)
            
            df = calcularMediaMovel(df)
            
            df = suavizacaoExponencialSimples(df)
            
            df = holtLinear(df)
            
            df = holtWinter(df)
        
            df = arima(df)
            
            df = sarima(df)
            
            result = calcularErro(df)
            
            dfResult = pd.DataFrame(result).T
            dfResult.columns = ['cd_empresa', 'cd_mercadoria', 'modelo', 'previsao', 'erroMM']
               
#dfResult
#fim = timeit.default_timer()
#print(f'Tempo de execução: {(fim - inicio)/60} minutos')

365573


C:\Users\Murilo\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()
C:\Users\Murilo\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\Murilo\Anaconda3\lib\site-packages\pmdarima\arima\auto.py:870: ModelFitWarning: Unable to fit ARIMA for order=(2, 0, 2) seasonal_order=(1, 0, 1, 12); data is likely non-stationary. (if you do not want to see these warnings, run with error_action="ignore")
  ModelFitWarning)
C:\Users\Murilo\Anaconda3\lib\site-packages\statsmodels\tsa\statespace\representation.py:375: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(

LinAlgError: SVD did not converge